In [2]:
import numpy as np

data_x = np.array([[0.014598, 0.014112], [0.014598, 0.016107131]])

data_sigma = np.array([[43584350.6], [43195861.8]])  


X_test=np.array([[0.014598, 0.014112], [0.014598, 0.016107131]])
y_test=np.array([[43584350.6], [43195861.8]]) 

print(data_x)
print(data_sigma)
assert data_x is not None, "data_x is None!"
assert data_sigma is not None, "data_sigma is None!"
print(type(data_x))  # Should print <class 'numpy.ndarray'>
print(type(data_sigma))  # Should print <class 'numpy.ndarray'>



import deepxde as dde

# Use the DataSet data object to create a data-driven task
data = dde.data.dataset.DataSet(X_train=data_x, y_train=data_sigma, X_test=X_test, y_test=y_test)

# Define the network for σ
layer_size = [2] + [50] * 3 + [1]  # 2 inputs (x, y ) -> hidden layers -> 1 output (σ)
activation = "tanh"
initializer = "Glorot uniform"
net_sigma = dde.nn.FNN(layer_size, activation,initializer)

# Define the model
model_sigma = dde.Model(data, net_sigma)

# Compile the model with data
model_sigma.compile("adam", lr=1e-3)

# Train the model
losshistory, train_state = model_sigma.train(epochs=5000)


[[0.014598   0.014112  ]
 [0.014598   0.01610713]]
[[43584350.6]
 [43195861.8]]
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>



Using backend: tensorflow.compat.v1
Other supported backends: tensorflow, pytorch, jax, paddle.
paddle supports more examples now and is recommended.



Instructions for updating:
non-resource variables are not supported in the long term
Compiling model...
Building feed-forward neural network...



c:\Users\yesda\Documents\GitHub\PINN_testing\.conda\Lib\site-packages\deepxde\nn\tensorflow_compat_v1\fnn.py:116: UserWarning: `tf.layers.dense` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Dense` instead.
  return tf.layers.dense(


'build' took 0.227050 s


'compile' took 0.367009 s

Training model...

Step      Train loss    Test loss     Test metric
0         [1.88e+15]    [1.88e+15]    []  
1000      [1.88e+15]    [1.88e+15]    []  
2000      [1.88e+15]    [1.88e+15]    []  
3000      [1.88e+15]    [1.88e+15]    []  
4000      [1.88e+15]    [1.88e+15]    []  
5000      [1.88e+15]    [1.88e+15]    []  

Best model at step 5000:
  train loss: 1.88e+15
  test loss: 1.88e+15
  test metric: []

'train' took 4.886712 s



In [2]:
def pde_C_L(x, C_L):
    dC_L_dt = dde.grad.jacobian(C_L, x, i=0, j=2)
    dC_L_dx2 = dde.grad.hessian(C_L, x, i=0, j=0)
    dC_L_dy2 = dde.grad.hessian(C_L, x, i=0, j=1)
    eq = dC_L_dt - (1/3.8) * (dC_L_dx2 + dC_L_dy2)
    return eq

In [5]:
x=np.array([[2,1,3],[3,1,4],[4,1,5]])
print(x[:,0:2])
sigma_A = model_sigma.predict(x[:,0:2])
print(sigma_A)
igma_A = model_sigma.predict([[2,1]])
print(igma_A)

[[2 1]
 [3 1]
 [4 1]]
[[267.8698]
 [267.8698]
 [267.8698]]
[[267.86975]]


In [11]:
# Define the square domain (e.g., 2x2 square with side length L)
L = 1.0  # Half-length of the square plate
square = dde.geometry.Rectangle([-L, -L], [L, L])

# Define the circular hole at the center (radius R)
R = 0.1  # Radius of the hole
circle = dde.geometry.Disk([0, 0], R)

# Geometry and time domain
geom = dde.geometry.csg.CSGDifference(square, circle)

timedomain = dde.geometry.TimeDomain(0, 1)
geomtime = dde.geometry.GeometryXTime(geom, timedomain)
def boundary_condition(x):
    # Extract points A and B (define these appropriately)
    data=np.array(x)
    x_A=np,array([2,1])
    x_B=np.array([1,2])
    if model_sigma is None:
        raise ValueError("model_sigma is not initialized properly.")
    # Apply the equation using sigma_model predictions
    sigma_A = model_sigma.predict(x_A)
    sigma_B = model_sigma.predict(x_B)
   
    if sigma_A is None or sigma_B is None:
        raise ValueError("The prediction returned None.")
    print(f"Predictions - sigma_A: {sigma_A}, sigma_B: {sigma_B}")
    
    # Boundary condition formula
    VH, R, T = 1.0, 1.0, 1.0  # Adjust constants accordingly
    exponent = (VH / (R * T)) * (sigma_A)
    
    return np.exp(exponent)  # This should be returned as a constraint

# Define boundary conditions within DeepXDE
def boundary_func(x, on_boundary):
    return boundary_condition(x) if on_boundary else None




# Apply the boundary condition defined earlier
bc = dde.DirichletBC(geomtime, boundary_func, lambda _, on_boundary: on_boundary)
ic = dde.icbc.IC(geomtime, lambda x: 20, lambda _, on_initial: on_initial)


data = dde.data.TimePDE(
    geomtime,
    pde_C_L,[ic,bc],
    num_domain=1000,
    num_boundary=150,
    num_initial=180,
    num_test=1000,
    )

# Network for C_L
layer_size = [3] + [64] * 3 + [1]  
activation = "tanh"
initializer = "Glorot uniform"
net_C_L = dde.nn.FNN(layer_size, activation, initializer)



# Define the model with the PDE, initial and boundary conditions
model= dde.Model(data, net_C_L)

# Compile the model with the PDE and dataset
model.compile("adam", lr=1e-3)

# Train the model
model.train(epochs=5000)
dde.saveplot(losshistory, train_state, issave=True, isplot=True)

Compiling model...
Building feed-forward neural network...
'build' took 0.029589 s



c:\Users\yesda\Documents\GitHub\PINN_testing\.conda\Lib\site-packages\deepxde\nn\tensorflow_compat_v1\fnn.py:116: UserWarning: `tf.layers.dense` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Dense` instead.
  return tf.layers.dense(


TypeError: 'NoneType' object is not subscriptable